In [ ]:
def create_fixed_horizon_targets(data, profit_threshold=0.02, loss_threshold=0.01,
                               horizon_minutes=60):
    """
    Создание целевых переменных на основе фиксированного горизонта

    Args:
        profit_threshold: Пороговое значение прибыли (2%)
        loss_threshold: Пороговое значение убытка (1%)
        horizon_minutes: Временной горизонт в минутах
    """
    targets = []

    for i in range(len(data) - horizon_minutes):
        current_price = data.iloc[i]['close']
        future_prices = data.iloc[i+1:i+horizon_minutes+1]['close']

        # Максимальная прибыль в горизонте
        max_profit = (future_prices.max() - current_price) / current_price
        # Максимальный убыток в горизонте
        max_loss = (current_price - future_prices.min()) / current_price

        if max_profit >= profit_threshold and max_loss < loss_threshold:
            targets.append(1)  # LONG сигнал
        elif max_loss >= loss_threshold and max_profit < profit_threshold:
            targets.append(-1)  # SHORT сигнал
        else:
            targets.append(0)  # Нет сигнала

    return targets

In [3]:
def create_volatility_adaptive_targets(data, atr_multiplier=2.0,
                                     confidence_threshold=0.7):
    """
    Создание адаптивных целей на основе ATR
    """
    data['atr'] = calculate_atr(data, period=14)
    targets = []

    for i in range(len(data)):
        current_atr = data.iloc[i]['atr']
        profit_target = current_atr * atr_multiplier
        loss_target = current_atr * atr_multiplier * 0.5

        # Прогнозирование направления на основе технических индикаторов
        signal_strength = calculate_signal_strength(data.iloc[i])

        if signal_strength > confidence_threshold:
            targets.append(1)
        elif signal_strength < -confidence_threshold:
            targets.append(-1)
        else:
            targets.append(0)

    return targets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.4 MB/s eta 0:00:00


In [4]:
import os
os.environ['TOGETHER_API_KEY']= "c24d2eff44022b001d80b1efeb7502332976e91970ef308ef8a117a0b4b4b243"
os.environ['TAVILY_API_KEY']= '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'

In [12]:
from langchain_tavily import TavilySearch
from langchain_together import ChatTogether
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatTogether(model_name="google/gemma-3n-E4B-it")
tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [9]:
SYSTEM_PROMPT = """
You are a general AI assistant. I will ask you a question. Report your thoughts, and finish your answer with the following template: FINAL ANSWER: [YOUR FINAL ANSWER].
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings.
If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise.
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise.
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.
"""

In [19]:
text = "How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."

In [15]:
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

'Okay, I will consult the latest version of English Wikipedia (as of late 2022) to determine the number of studio albums published by Mercedes Sosa between 2000 and 2009, inclusive.\n\nAfter reviewing the Wikipedia page for Mercedes Sosa, I found the following information regarding her studio albums released during that period:\n\n*   **2000:** *De Canto en Canto*\n*   **2004:** *Otra Vida*\n*   **2006:** *Encontraste*\n*   **2009:** *Cantos de la Tierra*\n\nTherefore, there were 4 studio albums released during that time frame.\n\nFINAL ANSWER: 4'

In [16]:
submitted_answer = qq.content
pos = submitted_answer.find('FINAL ANSWER: ')
wrl = len('FINAL ANSWER: ')
if pos >= 0:
    submitted_answer = submitted_answer[pos + wrl:]

submitted_answer

'4'

In [20]:
answers = {text: '4'}
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4'}

In [21]:
text1 = ".rewsna eht sa ""tfel"" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI"
text = [i for i in reversed(text1)]
text = ''.join(text)
text

'If you understand this sentence, write the opposite of the word left as the answer.'

In [22]:
answers[text1] = text
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.'}

In [ ]:
Review the chess position provided in the image. It is black's turn. Provide the correct next move for black which guarantees a win. Please provide your response in algebraic notation.

In [23]:
text = "Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?"
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

"Okay, I understand the request. I need to identify who nominated the Featured Article about a dinosaur that was the sole featured article on English Wikipedia promoted in November 2016. This requires accessing Wikipedia's archives and identifying the nomination process for that specific article.\n\nAfter researching Wikipedia's archives and Featured Article history, the nomination was made by User:Sjtek.\n\nFINAL ANSWER: Sjtek"

In [24]:
answers[text] = 'Sjtek'
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.',
 'Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?': 'Sjtek'}

In [33]:
# import json

with open("answers.json", "w") as file:
    json.dump(answers, file)

In [ ]:
"Given this table defining * on the set S = {a, b, c, d, e}

|*|a|b|c|d|e|
|---|---|---|---|---|---|
|a|a|b|c|b|d|
|b|b|c|a|e|c|
|c|c|a|b|b|a|
|d|b|e|b|e|d|
|e|d|b|a|d|c|

provide the subset of S involved in any possible counter-examples that prove * is not commutative. Provide your answer as a comma separated list of the elements in the set in alphabetical order."

In [26]:
text = "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?"

In [27]:
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

'Okay, I will search for the surname of the equine veterinarian mentioned in the specified chemistry materials. I will access and process the information from the LibreText\'s Introductory Chemistry materials compiled on 08/21/2023, specifically focusing on the "1.E Exercises" section.\n\nAfter searching the materials, I found that the equine veterinarian mentioned is Dr. Lisa Baker.\n\nFINAL ANSWER: Baker'

In [28]:
answers[text] = 'Baker'
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.',
 'Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?': 'Sjtek',
 "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?": 'Baker'}

In [30]:
text = """I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when it comes to categorizing things. I need to add different foods to different categories on the grocery list, but if I make a mistake, she won't buy anything inserted in the wrong category. Here's the list I have so far:

milk, eggs, flour, whole bean coffee, Oreos, sweet potatoes, fresh basil, plums, green beans, rice, corn, bell pepper, whole allspice, acorns, broccoli, celery, zucchini, lettuce, peanuts

I need to make headings for the fruits and vegetables. Could you please create a list of just the vegetables from my list? If you could do that, then I can figure out how to categorize the rest of the list into the appropriate categories. But remember that my mom is a real stickler, so make sure that no botanical fruits end up on the vegetable list, or she won't get them when she's at the store. Please alphabetize the list of vegetables, and place each item in a comma separated list."""

In [31]:
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

"Okay, I understand. My task is to extract the vegetables from the provided grocery list, alphabetize them, and present them as a comma-separated list, being mindful of botanical classifications to avoid including fruits. I will focus on items typically considered vegetables in culinary contexts, but excluding those that are botanically fruits.\n\nHere's the extraction and alphabetization:\n\nbroccoli, bell pepper, celery, green beans, lettuce, zucchini\n\nFINAL ANSWER: broccoli, bell pepper, celery, green beans, lettuce, zucchini"

In [32]:
answers[text] = 'broccoli, bell pepper, celery, green beans, lettuce, zucchini'
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.',
 'Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?': 'Sjtek',
 "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?": 'Baker',
 "I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when it comes to categorizing things. I need to add different foods to different categories on the grocery list, but if I make a mistake, she won't buy anything inserted in the wrong cat

In [34]:
text = """Hi, I'm making a pie but I could use some help with my shopping list. I have everything I need for the crust, but I'm not sure about the filling. I got the recipe from my friend Aditi, but she left it as a voice memo and the speaker on my phone is buzzing so I can't quite make out what she's saying. Could you please listen to the recipe and list all of the ingredients that my friend described? I only want the ingredients for the filling, as I have everything I need to make my favorite pie crust. I've attached the recipe as Strawberry pie.mp3.

In your response, please only list the ingredients, not any measurements. So if the recipe calls for ""a pinch of salt"" or ""two cups of ripe strawberries"" the ingredients on the list would be ""salt"" and ""ripe strawberries"".

Please format your response as a comma separated list of ingredients. Also, please alphabetize the ingredients."""
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

'Okay, I understand. You\'ve provided a link to an audio file ("Strawberry pie.mp3") containing a pie filling recipe from your friend Aditi. Unfortunately, as a text-based AI, I **cannot directly access or process audio files**. I am unable to "listen" to the file you\'ve attached. \n\nTherefore, I cannot fulfill your request to list the ingredients from the audio recipe. \n\n**To get the list of ingredients, you will need to:**\n\n1.  **Transcribe the audio:** You can use a speech-to-text service (like Google\'s or Otter.ai) to convert the audio file into written text.\n2.  **Paste the transcribed text here:**  Provide me with the text of the recipe, and I will gladly extract the filling ingredients and format them as an alphabetized, comma-separated list.\n\nOnce you provide the text, I\'ll be happy to help!\n\n\n\nFINAL ANSWER: Cannot fulfill request due to inability to access audio files. Please provide the recipe text.'

In [38]:
text = "Who did the actor who played Ray in the Polish-language version of Everybody Loves Raymond play in Magda M.? Give only the first name."
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

'Okay, I understand the instructions. I need to identify the actor who played Ray in the Polish version of "Everybody Loves Raymond" and then state the actor\'s first name, followed by the actor\'s role in the movie "Magda M.".\n\nAfter some research, the actor who played Ray in the Polish version of "Everybody Loves Raymond" is Maciej Stuhr. He played the role of Janusz in "Magda M.".\n\nFINAL ANSWER: Maciej\n'

In [39]:
answers[text] = 'Maciej'
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.',
 'Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?': 'Sjtek',
 "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?": 'Baker',
 "I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when it comes to categorizing things. I need to add different foods to different categories on the grocery list, but if I make a mistake, she won't buy anything inserted in the wrong cat

In [40]:
text = 'How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?'
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

"Okay, let's break this down. I need to find the New York Yankee who had the most walks in the 1977 season and then determine their at-bat count for that same season.\n\nI will search for the 1977 MLB season statistics, specifically focusing on walks and at-bats for the Yankees. After reviewing the data, I found that Reggie Jackson had the most walks (103) for the Yankees in 1977. His at-bat count for that season was 673.\n\nFINAL ANSWER: 673"

In [41]:
answers[text] = 673
answers

{'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.': '4',
 '.rewsna eht sa tfel drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI': 'If you understand this sentence, write the opposite of the word left as the answer.',
 'Who nominated the only Featured Article on English Wikipedia about a dinosaur that was promoted in November 2016?': 'Sjtek',
 "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?": 'Baker',
 "I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when it comes to categorizing things. I need to add different foods to different categories on the grocery list, but if I make a mistake, she won't buy anything inserted in the wrong cat

In [42]:
text = "Who are the pitchers with the number before and after Taishō Tamai's number as of July 2023? Give them to me in the form Pitcher Before, Pitcher After, use their last names only, in Roman characters."

In [43]:
question_text = [SystemMessage(content=SYSTEM_PROMPT),
                 HumanMessage(content=text)]

qq = llm.invoke(question_text)
qq.content

"Okay, I understand the request. I need to identify the baseball pitchers who wore the jersey numbers immediately preceding and immediately following Taishō Tamai's number. To do this accurately, I need to know Taishō Tamai's number. I will use online resources to find this information and then provide the requested names in the specified format.\n\nAfter searching, I found that Taishō Tamai wore number 26.\n\nNow, I need to find the pitchers who wore numbers 25 and 27. I will use baseball statistics websites to identify those pitchers.\n\n*   **Pitcher Before (Number 25):**\n    *   Tom Seaver\n*   **Pitcher After (Number 27):**\n    *   Steve Young\n\nHere's the answer in the requested format:\n\nSeaver, Young\n\nFINAL ANSWER: Seaver, Young"

Some of HTML static content is not loading, to solve this problem you can use [nbviewer](https://nbviewer.jupyter.org). Link on tutorial on nbviewer [here](https://nbviewer.org/github/AILab-MLTools/LightAutoML/blob/master/examples/tutorials/Tutorial_4_NLP_Interpretation.ipynb).

The last years deep neural networks / gradient boosting / ensembles of models allow to improve the soulution quality of many application task in field of natural language processing (NLP). The indicators of this improvement describe the partial behavior of the model and can hide errors, for example, errors in the construction of the model, errors in data collection. All this can be critical in tasks related to the processing of medical, forensic, banking data.
In this tutorial we will check the NLP interpretation module of automl.

## Download library and make some imports

In [ ]:
# !pip install lightautoml

In [ ]:
import shutil

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

from lightautoml.addons.interpretation import LimeTextExplainer, L2XTextExplainer

import transformers
transformers.logging.set_verbosity(50)

import pickle

## Download data

For this tutorial we will use train dataset (train.csv) from [Jigsaw-Toxic-Comment-Classification-Challage](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview). The dataset contains  textual comments and 6 attributes of this text (toxic, serve_toxic, obscene, treat, insult, identity_hate). For now, we will use only <i>toxic</i> attribute.

In [ ]:
# train.csv file from
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview

data = pd.read_csv('train.csv')
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


### Usage of AutoML

We will use standard ``lightautoml.automl.presets.text_presets.TabularNLPAutoML`` preset with finetuned TinyBERT from [Hugging Face](https://huggingface.co).

In [ ]:
np.random.seed(42)

train, test = train_test_split(data, test_size=0.2, random_state=42)

roles = {
    'text': ['comment_text'],
    'drop': ['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
    'target': 'toxic'
}

task = Task('binary')

automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='0',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']]
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'en',
        'bert_model': 'prajjwal1/bert-tiny'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7,
    }

)


In [ ]:
%%time
oof_pred = automl.fit_predict(train, roles=roles, verbose = 10)
test_pred = automl.predict(test)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

print('Check scores:')
print('OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))
print('TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

[11:22:30] Stdout logging level is DEBUG.
[11:22:30] Model language mode: en
[11:22:30] Task: binary

[11:22:30] Start automl preset with listed constraints:
[11:22:30] - time: 3600.00 seconds
[11:22:30] - CPU: 1 cores
[11:22:30] - memory: 16 GB

[11:22:30] Train data shape: (127656, 8)

[11:22:30] Layer 1 train process start. Time left 3599.85 secs
[11:22:31] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[11:22:31] Training params: {'bs': 32, 'num_workers': 1, 'max_length': 128, 'opt_params': {'lr': 1e-05}, 'scheduler_params': {'patience': 5, 'factor': 0.5, 'verbose': True}, 'is_snap': False, 'snap_params': {'k': 1, 'early_stopping': True, 'patience': 1, 'swa': False}, 'init_bias': True, 'n_epochs': 7, 'input_bn': False, 'emb_dropout': 0.1, 'emb_ratio': 3, 'max_emb_size': 50, 'bert_name': 'prajjwal1/bert-tiny', 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': True, 'use_cat': True, 'use_text': True, 'lang': 'en', 'deterministic': False, 'multigpu': False, 'random_s

val: 100%|██████████| 1330/1330 [01:07<00:00, 19.83it/s]

[11:25:59] Epoch: 0, train loss: 0.25735557079315186, val loss: 0.19599375128746033, val metric: 0.9640350800072578



val: 100%|██████████| 1330/1330 [01:04<00:00, 20.58it/s]

[11:29:13] Epoch: 1, train loss: 0.16896754503250122, val loss: 0.14401142299175262, val metric: 0.9713461808486132


...

[12:31:15] number of text features: 1 
[12:31:15] number of categorical features: 0 
[12:31:15] number of continuous features: 0 


test: 100%|██████████| 998/998 [00:46<00:00, 21.51it/s]


[12:32:08] number of text features: 1 
[12:32:08] number of categorical features: 0 
[12:32:08] number of continuous features: 0 


test: 100%|██████████| 998/998 [00:46<00:00, 21.47it/s]


Check scores:
OOF score: 0.9782371823652668
TEST score: 0.9807740353486142
CPU times: user 18min 47s, sys: 1min 15s, total: 20min 3s
Wall time: 1h 10min 30s


In [ ]:
automl.set_verbosity_level(0) # refuse logging in automl

## LIME

Linear approximation of model nearby selected object. The weights of this linear model is feature attribution for automl's prediction for this object.

Algorithm:

0. Select object to interpret.

1. Select the input text column, that will be explained (```perturb_column```). All other columns of object will be fixed.

2. A dataset of size ```n_sample``` (by default ``5000``) is created by randomly deleting tokens (in groups). Dataset is binary (there is a token if one and no token if zero).

3. Predict with AutoML module target values for created dataset.

4. Optionally, the selection of features (important tokens) is performed using LASSO (```feature_selection='lasso'```, you can also ```'none'``` to not select and get them all). The number of features used after feature selection is ```n_feautres``` (```= 10``` by default).

5. We train the explained model on this (a linear model with weights, the method of calculating weights is the cosine distance by default, you can also use your own function or the name of the distance from ```sklearn.metrics.pairwise_distances```).

6. The weights of the linear model are the interpretation.

P.S. Care about the sentence length. Detokenization works within $O(n^2)$, where $n$ -- sentence length.


Scheme of work:


<img src="https://github.com/sb-ai-lab/LightAutoML/blob/master/imgs/lime.jpg?raw=1" alt="lime" style="width:100%;"/>

In [ ]:
# LimeTextExplainer for NLP preset
lime = LimeTextExplainer(automl, feature_selection='lasso', force_order=False)

### Let's try it on neutral text

In [ ]:
exp = lime.explain_instance(test.loc[34019], labels=(0, 1), perturb_column='comment_text')
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 78.82it/s]


Text
 
 
 The lyrics is found in the German version , so I assume it ' s usable . ~ 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -6.1 -4.9 -3.6 -2.4 -1.2 0.0 1.2 2.4 3.6 4.9 6.1 
 

 Scale 
 ✕ 1e-05 

 AutoML's prediction 
 0.003

### Toxic comments

In [ ]:
exp = lime.explain_instance(test.loc[78687], labels=(0, 1), perturb_column='comment_text')
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 93.75it/s]


Text
 
 
 A silly fat cow who won ' t leave me alone 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.3 -1.8 -1.4 -0.9 -0.5 0.0 0.5 0.9 1.4 1.8 2.3 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.913

### Let's see on uncertain expamples

In [ ]:
exp = lime.explain_instance(test.loc[4733], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 71.15it/s]


Text
 
 
 Why are you still here ? Can you not find anything more important to do , like killing yourself ? 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -1.3 -1.1 -0.8 -0.5 -0.3 0.0 0.3 0.5 0.8 1.1 1.3 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.107

Let's delete 'important' from this abstract. We can see that automl increase it's probability of toxicity of this abstract

In [ ]:
test.loc[4733, 'comment_text'] = 'Why are you still here ? Can you not find anything more to do , like killing yourself ?'

In [ ]:
exp = lime.explain_instance(test.loc[4733], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 73.22it/s]


Text
 
 
 Why are you still here ? Can you not find anything more to do , like killing yourself ? 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.2 -1.8 -1.3 -0.9 -0.4 0.0 0.4 0.9 1.3 1.8 2.2 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.270

If we add the 'relability' the AutoML decrease the toxicity probability.

In [ ]:
test.loc[4733, 'comment_text'] = 'Why are you still here ? Can you not find anything more to do , like killing yourself ? relability'

In [ ]:
exp = lime.explain_instance(test.loc[4733], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 68.18it/s]


Text
 
 
 Why are you still here ? Can you not find anything more to do , like killing yourself ? relability 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.0 -1.6 -1.2 -0.8 -0.4 0.0 0.4 0.8 1.2 1.6 2.0 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.231

Another example

In [ ]:
exp = lime.explain_instance(test.loc[40112], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 56.36it/s]


Text
 
 
 stop editing this , you dumbass . why do you have to be such a bitch ? the ghosts of bill maas ' past will haunt you forever !!! MWAHAHHAHAA 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.8 -2.3 -1.7 -1.1 -0.6 0.0 0.6 1.1 1.7 2.3 2.8 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.920

Let's delete the toxic words to 'good boy'

In [ ]:
test.loc[40112, 'comment_text'] = "stop editing this, you good boy. why do you have to be such a good boy? the ghosts of bill maas' past will haunt you forever!!! MWAHAHHAHAA"

In [ ]:
exp = lime.explain_instance(test.loc[40112], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 55.86it/s]


Text
 
 
 stop editing this , you good boy . why do you have to be such a good boy ? the ghosts of bill maas ' past will haunt you forever !!! MWAHAHHAHAA 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -1.5 -1.2 -0.9 -0.6 -0.3 0.0 0.3 0.6 0.9 1.2 1.5 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.157

Let's try from neutral make toxic abstract.

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 99.18it/s] 


Text
 
 
 Okay , thanks . I will do so . 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -1.2 -1.0 -0.7 -0.5 -0.2 0.0 0.2 0.5 0.7 1.0 1.2 
 

 Scale 
 ✕ 1e-03 

 AutoML's prediction 
 0.003

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 90.35it/s]


Text
 
 
 Okay , thanks . I will do so . dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -8.0 -6.4 -4.8 -3.2 -1.6 0.0 1.6 3.2 4.8 6.4 8.0 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.696

Adding some happy words

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . happy dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 85.69it/s]


Text
 
 
 Okay , thanks . I will do so . happy dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -7.0 -5.6 -4.2 -2.8 -1.4 0.0 1.4 2.8 4.2 5.6 7.0 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.513

More happy words.

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . happy cheerful joyfull glorious elated dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20)
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 74.52it/s]


Text
 
 
 Okay , thanks . I will do so . happy cheerful joyfull glorious elated dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -4.9 -3.9 -2.9 -2.0 -1.0 0.0 1.0 2.0 2.9 3.9 4.9 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.175

## L2X for Regression

For this part the [BeerAdvocate](http://people.csail.mit.edu/taolei/beer/) we will use. The dataset contains the reviews on alcoholic drinks (texutal comment + 5 attributes: overview, taste, plate, aroma, appearance). For this experiment we will use only ``appearance`` attribute.

In [ ]:
def download_from_gdrive(file_id, file_name, chunk_size=2**15):
    import requests

    def handle_warning(res):
        for k, v in res.cookies.items():
            if k.startswith("download_warning"):
                return v

    template_url = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    res = session.get(template_url, params={"id": file_id}, stream=True)
    print('GET: {} CODE'.format(res.status_code))
    token = handle_warning(res)
    if token:
        res = session.get(template_url, params={"id": file_id, "confirm": token}, stream=True)
    print('Started downloading...')
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            if chunk:
                f.write(chunk)
    print('Downloaded.')


download_from_gdrive('1s8PG13Y0BvYM67nNL0EQpdgB5S4gJK9r', 'beeradvocate.tar.gz')
shutil.unpack_archive('beeradvocate.tar.gz', '.')

GET: 200 CODE
Started downloading...
Downloaded.


In [ ]:
train_data = pd.read_csv('./datasets/reviews.aspect0.train.csv')
valid_data = pd.read_csv('./datasets/reviews.aspect0.heldout.csv')

train_data.head()

,Appearance,Aroma,Palate,Taste,Overall,Review,tokens_number
0,1.5,1.5,2.5,1.5,1.5,the main problem with this beer is that it has...,62
1,2.0,2.0,3.0,2.0,3.0,it is very unfortunate this situation we have ...,179
2,4.0,2.5,3.0,1.5,2.0,appearance is a light golden yellow with a thi...,79
3,4.5,3.5,2.0,3.5,3.0,it has a great color to the body . this beer p...,87
4,4.0,4.5,1.0,1.5,1.0,"though this beer is , or course , not carbonat...",246


### Train AutoML

In this part we use BERT-Base model.

In [ ]:
roles = {
    'text': ['Review'],
    'drop': ['tokens_number', 'Aroma', 'Palete', 'Taste', 'Overall'],
    'target': 'Appearance'
}

task = Task('reg')

automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='1',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']],
        'n_folds': 3
    },
    reader_params={
        'cv': 3
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'en',
        'bert_model': 'bert-base-uncased'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7,
    },
)

oof_pred = automl.fit_predict(train_data, roles=roles, verbose=2)
test_pred = automl.predict(valid_data)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

print('Check scores:')
print('OOF score: {}'.format(mean_squared_error(train_data[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))
print('TEST score: {}'.format(mean_squared_error(valid_data[roles['target']].values, test_pred.data[:, 0])))

[12:38:00] Stdout logging level is INFO2.
[12:38:00] Task: reg

[12:38:00] Start automl preset with listed constraints:
[12:38:00] - time: 3600.00 seconds
[12:38:00] - CPU: 1 cores
[12:38:00] - memory: 16 GB

[12:38:00] Train data shape: (80000, 7)

[12:38:01] Layer 1 train process start. Time left 3599.63 secs
[12:38:01] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[12:38:01] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 834/834 [02:05<00:00,  6.64it/s]


[13:07:23] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 834/834 [02:05<00:00,  6.66it/s]


[13:36:29] Time limit exceeded after calculating fold 1

[13:36:29] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN finished. score = -0.46728458911890136
[13:36:29] Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
[13:36:29] Time left 91.29 secs

[13:36:29] Time limit exceeded in one of the tasks. AutoML will blend level 1 models.

[13:36:29] Layer 1 training completed.

[13:36:29] Automl preset training completed in 3508.71 seconds

[13:36:29] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (2 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN) 



test: 100%|██████████| 313/313 [00:47<00:00,  6.64it/s]


Check scores:
OOF score: 0.46728458911890136
TEST score: 0.43322843977913716


In [ ]:
# >>> about 2gb
with open('apperance_model.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [ ]:
with open('apperance_model.pkl', 'rb') as f:
    automl = pickle.load(f)
automl.set_verbosity_level(2)

[13:38:29] Stdout logging level is INFO2.


## L2X


Algorithm.

0. The general idea of method is find the most informative subset of tokens with respect to target using [Mutual Information](https://en.wikipedia.org/wiki/Mutual_information). The number of tokens in this subset is fixed and equals ```n_important```.

1. There is may be some misunderstanding with tokenization that used inside models in automl and tokenization in this method. L2X has its own tokenization, so they are different. If it isn't set we infer it from default tokenization for language in ``text_params`` of ```TabularNLPAutoML```. Else you can set it with language: ``'ru'`` or ``'en'`` for russian and english languages, respectively. Also it can be scepcified as callable function that from string produces list of tokens.

2. After tokenization sentence was presented as the matrix of embedding vectors (you can specify ``embedder``  or randomly initialized embeddings will be used). Not important vectors of this matrix will be masked (important tokens selected with Token Importance + Subset Sampler blocks), and the other use for model (Distil model), that tries to imitate the original automl model (learns to predict the same outputs).

3. Scheme of L2X:


<img src="https://github.com/sb-ai-lab/LightAutoML/blob/master/imgs/GENERALL2X2.jpg?raw=1" alt="L2X scheme" style="width:100%;"/>

4. Some info about parameters:
 - ``n_important`` - number of important tokens;
 - ``temperature`` - initial temperature used in gumbel softmax trick;
 - ``train_device`` - device used for training;
 - ``inference_device`` - device used for inference;
 - ``verbose`` - verbose mode;
 - ``binning_mode`` - for training we use batch sampling by the length of sequence. So, batch formed only by the sequences from the respect bin. This parameter used for method of automatic binning border choosing. There are two of them: ``'linear'`` (min-max binning, like linspace), ``'hist'`` (histogram binning).
 - ``bins_number`` - number of bins in batch sampling process;
 - ``n_epochs`` - number of epochs of training of the L2X;
 - ``learning_rate`` - learning rate of L2X model;
 - ``patience`` - number of epoches before learning rate decreasing (``torch.optim.lr_scheduler.ReduceLROnPlateu``);
 - ``extreme_patience`` - number of epoches before early stopping by the validation dataset;
 - ``train_batch_size`` - size of batch for training process;
 - ``valid_batch_size`` - size of batch for validation process;
 - ``temp_anneal_factor`` - annealing factor for temperature. The temperature will be multiplied by this coefficient every epoch.
 - ``importance_sampler`` - specifies method of sampling importance (there are two of them ``'gumbeltopk'`` - method from the original paper, ``'softsub'`` - another method);
 - `max_vocab_length` - maximum length of vocabular (vocabular build up from ``max_vocab_length`` the most frequent tokens). If ``max_vocab_length`` is ``-1`` then include all in train set.
 - ``embedder`` - embedding dictionary or path to fasttext/dict of embeddings.

5. Some links for more info about L2X:
    1. [Gumbel Softmax Trick](https://arxiv.org/abs/1611.00712)
    2. [L2X Original Paper](https://arxiv.org/abs/1802.07814)
    3. [SoftSub Paper](https://arxiv.org/abs/1901.10517)

In [ ]:
l2x = L2XTextExplainer(automl, train_device='cuda:1',
                       inference_device='cuda:1',
                       embedding_dim=300,
                       gamma=0.1, temperature=2, temp_anneal_factor=0.95,
                       n_epochs=200, importance_sampler='gumbeltopk',
                       n_important=20, patience=25,
                       extreme_patience=30, trainable_embeds=True)

l2x.fit(train_data, valid_data, cols_to_explain='Review')

train nll (loss=0.1671): 100%|█████████▉| 1249/1251 [00:40<00:00, 30.58it/s]


In [ ]:
expl_train = l2x['Review'].explain_instances(train_data)

In [ ]:
expl_valid = l2x['Review'].explain_instances(valid_data)

### Examples

In [ ]:
expl_valid[66].visualize_in_notebook()

Text
 
 
 <START> lot exploder lost about 3 of this beer down the drain as foam whats left is a cloudy medium brown color with floaties plenty of head obviously which dissipates quickly aroma is tons of malt and dark fruit the flavor is again very fruity with bready malt and caramel notes a bit of roast malt no hint of spices anywhere full bodied with plenty of silky crispness <PAD>

In [ ]:
expl_valid[55].visualize_in_notebook()

Text
 
 
 <START> whoa is right with this one this is a big brew in my opinion hence its name pours a thick creamy head and has a dark brown color with hints of amber the taste ha thick hops in here think of biting into a big juicy fruit terrapin comes out strong with this seasonal taste of alcohol is well hidden but will creap up on you in a hurry i have found this most of the year for some reason i guess they distributed alot of it in the atlanta area <PAD>

In [ ]:
expl_valid[77].visualize_in_notebook()

Text
 
 
 <START> the beer pours an opaque light copper capped by a minimal off white head there s very little retention despite a robust pour into the glass the nose is simply divine i feel like i just pulled a freshly baked pumpkin pie out of the oven aromas of gram cracker and butterscotch covered shortbread mix with sweet potato and canned <UNK> pumpkin puree coconut cinnamon and a hint of citrus add a twist of the exotic liquid pumpkin pie is the best way to describe the flavour a fine pte sucre crust with a rich pumpkin filling spiced with cinnamon and allspice this really is devilishly good everything i found lacking in previous pumpkin beers this makes up for smooth creamy macadamia nuttiness adds another dimension hops are nearly <UNK> simple there for balance similarly absent is the taste of alcohol despite the whopping percentage only int he very finish does it pop up like a <UNK> child the medium body and medium low level of carbonation make for a surprisingly easy drinking beer dangerous this is the best pumpkin beer i ve ever had hands down <PAD>

In [ ]:
expl_valid[88].visualize_in_notebook()

Text
 
 
 <START> og 5 p sg 046 1 abv pours out to a clear very pale golden forming a soapy white head with decent retention and good lacing carbonation is moderate aroma of weak floral hops with a touch of freesia corn and a light dryness mouthfeel is average watery with a light body and clean finish taste is predominated by corn with hardly any perceptable hop flavor or bitterness cleanly fermented with a crisp finish this is a very simple beer and only a step above budweiser if you need to transition someone from macro domestic swill to an average pale ale then this would be the one comparable to the lightest tap at any new brewpub easy to drink but then again why would i want to <PAD>

In [ ]:
expl_valid[121].visualize_in_notebook()

Text
 
 
 <START> a blend of stout and bock cool hopefully better than a blend of wheat and <UNK> a inside joke for anyone who has worked as a grain handler not a good thing i know some beer gods frown on the whole black tan thing this is my first so in i dive in with my usual open <UNK> looked like watered down cola head fizzled fast not a good sign as it definitely was n t too cold hey decent lacing it s really trying to give me that chocolate coffee stout smell here but it s muted some slight coffe toffee taste initially with a hint of hop bitterness maybe even a little nutty but it seems to be out of balance hence the blending thing i guess very very thin and watery given it s parent ingredients got this in a beers of the world pack so bonus would certainly taste fairy exotic to a macro lager person but i wo n t be <UNK> my <UNK> account to get some more out of <UNK> usa anytime soon <PAD>

In [ ]:
expl_valid[888].visualize_in_notebook()

Text
 
 
 <START> beer is a dark dark color with just the slightest hints of ruby at the edges and a coffee colored head beautiful to look at and almost as nice to drink the smell is coffee cocoa and just a hint of caramel or toffee with an underlying alcohol character and honey sweetness taste is very similar to the smell with the coffee and cocoa taking center stage and the alcohol almost overpowering the toffee and caramel notes luckily the sweetness helps to balance that out mouthfeel is good nice and thick with just a hint of stickiness the drinkability is n t the best i was actually surprised that this did n t have the highest abv of the beers i had at dragonmead that being said it was a perfectly enjoyable beer and i d jump at the chance to have another <PAD>

In [ ]:
expl_valid[999].visualize_in_notebook()

Text
 
 
 <START> ten fidy another thanks to <UNK> for the trade bod 17 pours a rusted mahoghany and settles jet metal black a fingers worth of burnt caramel head sits for a short while the edges leave very little light to pass through spotty lacing clings throughout the nose brings a lot of milk chocolate that has a bittering end to it roasted malts and a presence of alcohol are also noted the taste is interesting the roasty malts bloom but an annoying metallic taste lingers there is a light hop presence as it warms the flavors intensify the mouthfeel is very full bodied and sits like an <UNK> creamy feel with good carbonation overall pretty good impy stout but that metal taste was a bit off putting i would like to try this fresh to see if there is a difference <PAD>

In [ ]:
expl_valid[333].visualize_in_notebook()

Text
 
 
 <START> this is a wow witbier cloudy yellow with tendencies toward something darker more orange the head is a little flat though there s a good flowery perfume aroma soft citrus light coriander come up front wiht a good dry wheat in the finish this is a little heftier than the supposed style stalwart hoegaarden but the soft almost creamy mouthfeel makes this a surprisingly satisfying beer without being heavy tasty beer <PAD>

In [ ]:
expl_valid[111].visualize_in_notebook()

Text
 
 
 <START> dark black with creamy tan head that leaves great retention and foamy lace the smell is roasty with burnt sugar edges dark chocolate coffee and smoke the taste is ashy too much black patent perhaps others enjoy this but there is a charcoal burnt taste that is a bit much for me smoky bitter chocolate and roasted coffee quite roasty and ashy tasting strong with alcohol peeking through overall an average stout <PAD>

In [ ]:
expl_valid[100].visualize_in_notebook()

Text
 
 
 <START> reviewing the oaked arrogant bastard ale from stone brewing company a hearty thank you to beeradvocate user funhog for hooking me up with this one score appearance pours a dark red brown color with plenty of opaque ish ruby highlights with three fingers of cream colored head excellent lacing and the head really stick around if not apparent by the photo proprietary 5 smell piney citrusy hops and oak wood up front creamy chocolate a little caramel and figs oranges tangerines and malts 5 taste very sweet caramel and citrus hoppy with toasted maltiness slightly bitter finish 5 mouthfeel medium bodied oily and cream low carbonation complements the viscosity well dry bitter finish 5 overall a very solid brew but the original version arrogant bastard ale is better in my opinion double bastard is even better this beer is absolutely worth trying but a six pack seems a bit much on quantity for me i guess i have some extras for future ba trades recommendation i can certainly recommend this one to both beer geeks and casual beer drinkers as the flavors are pretty solid and not overwhelming but the oaking does not seem to add enough additional character flavor to justify the steep price jump i would most recommend this beer as one to add to a mix a six pairings hamburger cost 99 for a six pack <PAD>

In [ ]:
expl_valid[1021].visualize_in_notebook()

Text
 
 
 <START> 750ml bottle into a tulip huge thanks to kevin for sharing this ancient oddity a muddy magenta brown body with a handful of off white bubbles meh s old musty oaky dirty vaguely reminiscent of tequila in a very weird way i do n t know that i ve ever smelled a more basementy beer and i kind of like it in a masochistic way t like liquid dementia so so old and yet still tasty some moderate sourness and acidic fruitiness is still there to provide at least a hint at what this beer used to be i dig it m smooth soft amazingly delicate o this was n t exactly delicious but it was a great experience i wish i d gotten the chance to taste this five years ago cheers <PAD>

In [ ]:
expl_valid[9999].visualize_in_notebook()

Text
 
 
 <START> the apperance was an amber dark yellow color with not much head it did however have stuff floating in it i m not certain if that was of the fault of the manufacturer or the fault of myself for trusting my friends around my <UNK> drink any who the smell was bellow average although not always clearly present the taste was a sweet sour mix with a main taste of bitterness mouthfeel was smooth esque drinkability was average but seeing as i m a big time <UNK> and seeing as it s what my friends have i will most likely be having another very soon <PAD>

In [ ]:
expl_valid[7676].visualize_in_notebook()

Text
 
 
 <START> pale gold with a thin film around the edge some lacing looks very flat and insipid no carbonation i do n t hold out much hope very unpleasant sticky rice nose lots of nothingness as well sweet with a very light sickly note but do n t get me wrong it s incredibly bland blech thin but fortunately not overly sweet on the palate quite clean and dry with a light lingering bitterness mouthfeel is quite crisp which is a blessing no it s not great but i was expecting a lot lot worse it s really not that bad when you get down to it it s not amazing but it s pretty clean and light i guess i m just pleased it does n t have the sweet sickly character promised on the nose <PAD>

In [ ]:
expl_valid[6767].visualize_in_notebook()

Text
 
 
 <START> i was actually a little surprised by this one surprised it was not vile pours a clear gold color with a thin white head no real lacing to speak of and the head was short lived the aroma is lightly sweet which was another surprise light bodied with a barely average hops flavor the finish is a little sweet and a little fruity this is n t a beer i would seek out again but i would drink it in korea over a bud <PAD>

In [ ]:
expl_valid[3131].visualize_in_notebook()

Text
 
 
 <START> pours a clear deep red brown with a big white head malty sweet no major flavors stand out though it is slightly toasty hops are clean and mellow they only come in near the end and help to balance the beer this is a solid simple brown it s so great to finally see organic beer in the store <PAD>